In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
import uvicorn
from typing import List, Dict, Optional
import json
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI app
app = FastAPI(
    title="OPTIMAL PATH Refugee Resettlement API",
    description="AI-powered refugee resettlement optimization system",
    version="1.0.0",
    docs_url="/",
    redoc_url="/docs"
)

# Pydantic models
class RefugeeProfile(BaseModel):
    name: Optional[str] = "Anonymous"
    languages: List[str]
    job_skills: List[str]
    education_level: str
    family_size: int
    cultural_background: str
    special_needs: Optional[List[str]] = None
    health_requirements: Optional[List[str]] = None
    mental_health_support: bool = False

class CountryRecommendation(BaseModel):
    country: str
    confidence: float
    rank: int

class RecommendationResponse(BaseModel):
    recommended_country: str
    confidence_score: float
    match_breakdown: Dict[str, float]
    alternative_countries: List[CountryRecommendation]
    explanation: str
    utilization_warning: Optional[str] = None

class SystemStats(BaseModel):
    total_recommendations: int
    countries_used: List[str]
    average_confidence: float

# Mock country data (replace with your trained model)
COUNTRIES = ['USA', 'Canada', 'Germany', 'France', 'UK', 'Sweden', 'Australia']
COUNTRY_CAPACITIES = {
    'USA': 50000, 'Canada': 40000, 'Germany': 80000, 
    'France': 30000, 'UK': 25000, 'Sweden': 20000, 'Australia': 18000
}

# Track recommendations
recommendation_history = []
country_allocations = {country: 0 for country in COUNTRIES}

def calculate_match_scores(profile: RefugeeProfile) -> Dict[str, float]:
    """Calculate match scores between refugee profile and countries"""
    match_scores = {}
    
    for country in COUNTRIES:
        # Simulate AI model predictions
        base_score = np.random.uniform(0.6, 0.9)
        
        # Enhance based on profile features
        if 'English' in profile.languages and country in ['USA', 'Canada', 'UK', 'Australia']:
            base_score += 0.1
        if 'French' in profile.languages and country == 'France':
            base_score += 0.1
        if 'German' in profile.languages and country == 'Germany':
            base_score += 0.1
            
        # Job market matching
        skilled_jobs = ['IT', 'engineering', 'healthcare', 'education']
        if any(skill in skilled_jobs for skill in profile.job_skills) and country in ['USA', 'Germany', 'Canada']:
            base_score += 0.05
            
        # Family support
        if profile.family_size > 2 and country in ['Canada', 'Sweden', 'Germany']:
            base_score += 0.05
            
        # Healthcare needs
        if profile.health_requirements and country in ['Germany', 'Sweden', 'France']:
            base_score += 0.05
            
        # Capacity consideration
        utilization = country_allocations[country] / COUNTRY_CAPACITIES[country]
        if utilization > 0.8:
            base_score -= 0.2  # Penalize over-utilized countries
        elif utilization < 0.3:
            base_score += 0.1  # Reward under-utilized countries
            
        match_scores[country] = min(0.95, max(0.3, base_score))
    
    return match_scores

def generate_explanation(profile: RefugeeProfile, country: str, match_breakdown: Dict[str, float]) -> str:
    """Generate human-readable explanation for the recommendation"""
    explanations = []
    
    if match_breakdown.get('language_match', 0) > 0.7:
        explanations.append("strong language compatibility")
    if match_breakdown.get('job_match', 0) > 0.7:
        explanations.append("excellent job opportunities")
    if profile.family_size > 2 and match_breakdown.get('family_support', 0) > 0.6:
        explanations.append("comprehensive family support services")
    if profile.mental_health_support and match_breakdown.get('service_match', 0) > 0.6:
        explanations.append("robust mental health services")
    
    if explanations:
        return f"{country} is recommended because of {', '.join(explanations)}."
    else:
        return f"{country} offers the best overall balance of services and opportunities for your profile."

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "service": "OPTIMAL PATH Refugee Resettlement API",
        "version": "1.0.0",
        "timestamp": datetime.now().isoformat(),
        "countries_available": len(COUNTRIES)
    }

@app.get("/countries")
async def get_countries_info():
    """Get information about available countries"""
    country_info = []
    for country in COUNTRIES:
        utilization = country_allocations[country] / COUNTRY_CAPACITIES[country]
        country_info.append({
            "country": country,
            "capacity": COUNTRY_CAPACITIES[country],
            "current_allocations": country_allocations[country],
            "utilization": f"{utilization:.1%}",
            "available_capacity": COUNTRY_CAPACITIES[country] - country_allocations[country]
        })
    
    return {
        "countries": country_info,
        "total_capacity": sum(COUNTRY_CAPACITIES.values()),
        "total_allocations": sum(country_allocations.values())
    }

@app.post("/recommend", response_model=RecommendationResponse)
async def recommend_country(profile: RefugeeProfile):
    """Get country recommendation for a refugee profile"""
    try:
        # Calculate match scores
        match_scores = calculate_match_scores(profile)
        
        # Get top recommendation
        recommended_country = max(match_scores.items(), key=lambda x: x[1])[0]
        confidence = match_scores[recommended_country]
        
        # Generate match breakdown
        match_breakdown = {
            "job_match": np.random.uniform(0.6, 0.9),
            "language_match": np.random.uniform(0.5, 0.9),
            "community_match": np.random.uniform(0.5, 0.8),
            "service_match": np.random.uniform(0.6, 0.9),
            "family_support": np.random.uniform(0.5, 0.8) if profile.family_size > 1 else 0.3
        }
        
        # Get alternative countries
        sorted_countries = sorted(match_scores.items(), key=lambda x: x[1], reverse=True)
        alternative_countries = []
        for i, (country, score) in enumerate(sorted_countries[1:4], 2):  # Skip first (recommended)
            alternative_countries.append(
                CountryRecommendation(
                    country=country,
                    confidence=score,
                    rank=i
                )
            )
        
        # Generate explanation
        explanation = generate_explanation(profile, recommended_country, match_breakdown)
        
        # Check for utilization warning
        utilization = country_allocations[recommended_country] / COUNTRY_CAPACITIES[recommended_country]
        utilization_warning = None
        if utilization > 0.85:
            utilization_warning = f"Note: {recommended_country} currently has high utilization ({utilization:.1%})"
        
        # Update allocations and history
        country_allocations[recommended_country] += 1
        recommendation_history.append({
            "timestamp": datetime.now().isoformat(),
            "profile": profile.dict(),
            "recommendation": recommended_country,
            "confidence": confidence
        })
        
        return RecommendationResponse(
            recommended_country=recommended_country,
            confidence_score=confidence,
            match_breakdown=match_breakdown,
            alternative_countries=alternative_countries,
            explanation=explanation,
            utilization_warning=utilization_warning
        )
        
    except Exception as e:
        logger.error(f"Error generating recommendation: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/stats")
async def get_system_stats():
    """Get system statistics"""
    if not recommendation_history:
        return SystemStats(
            total_recommendations=0,
            countries_used=[],
            average_confidence=0.0
        )
    
    confidences = [r['confidence'] for r in recommendation_history]
    countries_used = list(set(r['recommendation'] for r in recommendation_history))
    
    return SystemStats(
        total_recommendations=len(recommendation_history),
        countries_used=countries_used,
        average_confidence=sum(confidences) / len(confidences)
    )

@app.get("/history")
async def get_recommendation_history(limit: int = 10):
    """Get recent recommendation history"""
    return {
        "history": recommendation_history[-limit:],
        "total_recommendations": len(recommendation_history)
    }

@app.post("/reset")
async def reset_system():
    """Reset the system (for testing)"""
    global country_allocations, recommendation_history
    country_allocations = {country: 0 for country in COUNTRIES}
    recommendation_history = []
    return {"message": "System reset successfully"}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=True)